<a href="https://colab.research.google.com/github/welcomeglory/Python/blob/master/student_grade.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
from google.colab import files
uploaded = files.upload()

Saving Student_performance_data _.csv to Student_performance_data _ (1).csv


In [9]:
import pandas as pd
import numpy as np
import tensorflow as tf
print(tf.__version__)
from tqdm import tqdm
from sklearn.model_selection import train_test_split
# 학생 등급 출력
# Define the model class # 32층
class StudentGradeANN(tf.keras.Model):
    def __init__(self):
        super(StudentGradeANN, self).__init__()

        self.hid1 = tf.keras.layers.Dense(32,
                                          activation='relu',
                                          kernel_initializer=self.weight_initialization(),
                                          bias_initializer=self.weight_initialization())
        self.drop1 = tf.keras.layers.Dropout(0.2)

        self.hid2 = tf.keras.layers.Dense(32,
                                          activation='relu',
                                          kernel_initializer=self.weight_initialization(),
                                          bias_initializer=self.weight_initialization())
        self.drop2 = tf.keras.layers.Dropout(0.2)

        self.hid3 = tf.keras.layers.Dense(32,
                                          activation='relu',
                                          kernel_initializer=self.weight_initialization(),
                                          bias_initializer=self.weight_initialization())
        self.drop3 = tf.keras.layers.Dropout(0.2)

        # 결과는 5개
        self.result = tf.keras.layers.Dense(5,
                                            activation=None,
                                            kernel_initializer=self.weight_initialization(),
                                            bias_initializer=self.weight_initialization())

    def weight_initialization(self):
        return tf.keras.initializers.RandomNormal(mean=0., stddev=1.)

    def call(self, x, training=False):  # Forward pass
        y1 = self.hid1(x)
        d1 = self.drop1(y1, training=training)
        y2 = self.hid2(d1)
        d2 = self.drop2(y2, training=training)
        y3 = self.hid3(d2)
        d3 = self.drop3(y3, training=training)
        logits = self.result(d3)
        sy = tf.nn.softmax(logits)
        return logits, sy

2.17.0


In [10]:
# Loss function
@tf.function
def cross_entropy_loss(logits, y):
    return tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=logits, labels=y))

# Optimizer and backward pass
learningRate = 0.001
optimizer = tf.optimizers.Adagrad(learningRate)

@tf.function
def backward(model, x, y):
    with tf.GradientTape() as grad:
        logits, sy = model(x, training=True)
        loss = cross_entropy_loss(logits, y)
    grads = grad.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(grads, model.trainable_variables))
    return loss

# Accuracy function
@tf.function
def accuracy(predY, y):
    correct_predictions = tf.equal(tf.argmax(predY, axis=1), tf.argmax(y, axis=1))
    acc = tf.reduce_mean(tf.cast(correct_predictions, tf.float32))
    return acc


In [11]:
# Training function
def train_learning(epochs, frequency, trainDataIter, model, XTest, yTest):
    y_loss = []
    for epoch in tqdm(range(epochs)):
        bx, by = next(trainDataIter)
        loss = backward(model, bx, by)
        y_loss.append(loss)
        if epoch % frequency == 0:
            predY, sy = model(bx, training=False)
            acc = accuracy(predY, by)
            print(f"Epoch: {epoch+1} ===> Loss: {loss:.4f}, Accuracy: {acc:.4f}")
    predY, sy = model(XTest, training=False)
    acc = accuracy(predY, yTest)
    print(f"Test Accuracy: {acc:.4f}")
    return y_loss

In [12]:
df = pd.read_csv("Student_performance_data _ (1).csv")
X = df[['Ethnicity', 'ParentalEducation',
       'StudyTimeWeekly', 'Absences', 'Tutoring', 'ParentalSupport',
       'Extracurricular', 'Sports', 'Music', 'Volunteering', 'GPA']].values
y = df[['GradeClass']].values
# 정규화
X = (X - X.min(axis=0)) / (X.max(axis=0) - X.min(axis=0))
# 훈련데이터, 테이트 데이터
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=7777)
# 원핫인코딩
y_train = tf.one_hot(y_train.flatten(), 5)
y_test = tf.one_hot(y_test.flatten(), 5)

train_data = tf.data.Dataset.from_tensor_slices((X_train, y_train))
train_data = train_data.shuffle(buffer_size=X_train.shape[0]).batch(300).repeat()
trainDataIter = iter(train_data)
bx, by = next(trainDataIter)
# print(bx.shape, by.shape)

epochs = 50000
frequency = 300

model = StudentGradeANN()
y_loss = train_learning(epochs, frequency, trainDataIter, model, X_test, y_test)

  0%|          | 1/50000 [00:01<26:56:22,  1.94s/it]

Epoch: 1 ===> Loss: 194.9201, Accuracy: 0.1667


  1%|          | 347/50000 [00:03<02:37, 314.78it/s]

Epoch: 301 ===> Loss: 100.3391, Accuracy: 0.1733


  1%|▏         | 661/50000 [00:04<02:10, 377.22it/s]

...

Epoch: 49501 ===> Loss: 9.5732, Accuracy: 0.4800


100%|█████████▉| 49833/50000 [02:37<00:00, 218.21it/s]

Epoch: 49801 ===> Loss: 11.8443, Accuracy: 0.4167


100%|██████████| 50000/50000 [02:38<00:00, 315.84it/s]


Test Accuracy: 0.4485
